In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from transformer import TransformerConfig
from lm import LM

from data import Dataset

In [42]:
dataset = Dataset()

config = TransformerConfig(d_model=32, n_layers=1, n_heads=1, max_len=dataset.max_len, dropout=0.)
model = LM(config, vocab_size=len(dataset.vocabulaire))
optim = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [43]:
sum(p.numel() for p in model.parameters())

19456

In [56]:
for i in range(10000):
    X, Y = dataset.get_batch('train', 32) # (B, L)
    logits = model(X) # (B, L, vocab_size)

    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=dataset.char_to_int['<pad>'])
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if i%1000==0:
        print(loss)

tensor(1.8989, grad_fn=<NllLossBackward0>)
tensor(1.7855, grad_fn=<NllLossBackward0>)
tensor(1.7722, grad_fn=<NllLossBackward0>)
tensor(1.9071, grad_fn=<NllLossBackward0>)
tensor(1.9395, grad_fn=<NllLossBackward0>)
tensor(1.6827, grad_fn=<NllLossBackward0>)
tensor(1.7053, grad_fn=<NllLossBackward0>)
tensor(1.7210, grad_fn=<NllLossBackward0>)
tensor(1.6573, grad_fn=<NllLossBackward0>)
tensor(1.7409, grad_fn=<NllLossBackward0>)


In [64]:
X, Y = dataset.get_batch('val', 256) # (B, L)
logits = model(X) # (B, L, vocab_size)

loss = F.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=dataset.char_to_int['<pad>'])
loss

tensor(1.8247, grad_fn=<NllLossBackward0>)

In [65]:
torch.save(model.state_dict(), "transformer_d_model32.pth")

In [35]:
# transformer_d_model16 : loss de 1.85
